In [39]:
import pandas as pd

import datetime as dt
import geopandas as gpd
from fiona.crs import from_epsg
import pyproj
import pylab as pl
import mplleaflet
import folium
#import shapefile as shp
import math
import branca.colormap as cm
import os
from shapely.geometry import Point
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Applications/anaconda/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['cm']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
#Pulling Data via API
# Read File
wifi_data = pd.read_json("http://wifindproject.com/wifipulling/?device_model=VS425&timeformat=1")

In [15]:
wifi_data.head()

,acc,altitude,app_version,bssid,caps,device_mac,device_model,droid_version,freq,idx,lat,level,lng,ssid,time
0,19.957,0.0,0.3.3,1c:e8:5d:ca:8f:33,[wpa-eap-tkip][wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2437,56629534,40.690966,-60,-73.987686,nyu-legacy,06-12-2017 15:49:53
1,19.957,0.0,0.3.3,1c:e8:5d:ca:8f:32,[wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2437,56629535,40.690966,-59,-73.987686,eduroam,06-12-2017 15:49:53
2,19.957,0.0,0.3.3,cc:d5:39:27:f2:43,[wpa-eap-tkip][wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2412,56629536,40.690966,-66,-73.987686,nyu-legacy,06-12-2017 15:49:53
3,19.957,0.0,0.3.3,cc:d5:39:27:f2:42,[wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2412,56629537,40.690966,-67,-73.987686,eduroam,06-12-2017 15:49:53
4,19.957,0.0,0.3.3,00:a2:ee:03:e3:20,[wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2412,56629538,40.690966,-73,-73.987686,constitution,06-12-2017 15:49:53


In [4]:
wifi_data = wifi_data.drop(wifi_data[wifi_data.acc > 50].index)

In [5]:
wifi_data.head()

,acc,altitude,app_version,bssid,caps,device_mac,device_model,droid_version,freq,idx,lat,level,lng,ssid,time
0,19.957,0.0,0.3.3,1c:e8:5d:ca:8f:33,[wpa-eap-tkip][wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2437,56629534,40.690966,-60,-73.987686,nyu-legacy,06-12-2017 15:49:53
1,19.957,0.0,0.3.3,1c:e8:5d:ca:8f:32,[wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2437,56629535,40.690966,-59,-73.987686,eduroam,06-12-2017 15:49:53
2,19.957,0.0,0.3.3,cc:d5:39:27:f2:43,[wpa-eap-tkip][wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2412,56629536,40.690966,-66,-73.987686,nyu-legacy,06-12-2017 15:49:53
3,19.957,0.0,0.3.3,cc:d5:39:27:f2:42,[wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2412,56629537,40.690966,-67,-73.987686,eduroam,06-12-2017 15:49:53
4,19.957,0.0,0.3.3,00:a2:ee:03:e3:20,[wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2412,56629538,40.690966,-73,-73.987686,constitution,06-12-2017 15:49:53


In [6]:
# convert Unix timestamp into readable timestamp
#df['time2'] = map(lambda x: dt.datetime.fromtimestamp(x), df.time.astype(float)/1000)
#df['month'] = map(lambda x: x.month, df['time2'])
#df['day'] = map(lambda x: x.day, df['time2'])
#df['hour'] = map(lambda x: x.hour, df['time2'])
#df['minute'] = map(lambda x: x.minute, df['time2'])
#df['sec'] = map(lambda x: x.second, df['time2'])

In [18]:
# geo
#wifi_data.reset_index(drop=True, inplace=True)
wifi_data['geo'] = zip(wifi_data.lng, wifi_data.lat)
wifi_data['geometry'] = map(lambda x: Point(x), wifi_data['geo'])

In [31]:
#original_data["geometry"] = "POINT (" + original_data.lng + " " + original_data.lat + ")"
wifi_data["geometry"] = " "

In [34]:
for n, p in enumerate(zip(wifi_data.lng, wifi_data.lat)):
    wifi_data["geometry"].iloc[n] = Point(p)

/Applications/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [29]:
#del wifi_data["geometry"]
#del wifi_data["geo"]

In [24]:
wifi_data.lat.dtype

dtype('float64')

In [35]:
wifi_data.head()

,acc,altitude,app_version,bssid,caps,device_mac,device_model,droid_version,freq,idx,lat,level,lng,ssid,time,geometry
0,19.957,0.0,0.3.3,1c:e8:5d:ca:8f:33,[wpa-eap-tkip][wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2437,56629534,40.690966,-60,-73.987686,nyu-legacy,06-12-2017 15:49:53,POINT (-73.987686 40.6909661)
1,19.957,0.0,0.3.3,1c:e8:5d:ca:8f:32,[wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2437,56629535,40.690966,-59,-73.987686,eduroam,06-12-2017 15:49:53,POINT (-73.987686 40.6909661)
2,19.957,0.0,0.3.3,cc:d5:39:27:f2:43,[wpa-eap-tkip][wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2412,56629536,40.690966,-66,-73.987686,nyu-legacy,06-12-2017 15:49:53,POINT (-73.987686 40.6909661)
3,19.957,0.0,0.3.3,cc:d5:39:27:f2:42,[wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2412,56629537,40.690966,-67,-73.987686,eduroam,06-12-2017 15:49:53,POINT (-73.987686 40.6909661)
4,19.957,0.0,0.3.3,00:a2:ee:03:e3:20,[wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2412,56629538,40.690966,-73,-73.987686,constitution,06-12-2017 15:49:53,POINT (-73.987686 40.6909661)


In [40]:
# groupby geo, unique bssid
access_count = wifi_data.groupby(wifi_data.geometry).apply(lambda x: len(x.bssid.unique()))
access_bssidList = wifi_data.groupby(wifi_data.geometry).apply(lambda x: list(x.bssid.unique()))
df3 = pd.DataFrame(map(lambda x: Point(x), access_count.index), columns=['geometry'])
df3['unique_bssid_count'] = access_count.values
df3['unique_bssid_list'] = access_bssidList.values

TypeError: unhashable type: 'Point'

In [ ]:
# groupby geo, unique bssid
access_count = df2.groupby(df2.geo).apply(lambda x: len(x.bssid.unique()))
access_bssidList = df2.groupby(df2.geo).apply(lambda x: list(x.bssid.unique()))
df3 = pd.DataFrame(map(lambda x: Point(x), access_count.index), columns=['geometry'])
df3['unique_bssid_count'] = access_count.values
df3['unique_bssid_list'] = access_bssidList.values

# crs
df3= gpd.GeoDataFrame(df3)
df3.crs = from_epsg(4326)
df3.to_crs(epsg=2263, inplace=True)
df3.to_pickle('unique_bssid.p')

# grid boundry
all_x = map(lambda p: p.x, df3.geometry)
all_y = map(lambda p: p.y, df3.geometry)
minx, maxx, miny, maxy = min(all_x), max(all_x), min(all_y), max(all_y) 

# grid length
dx = cell_length
dy = cell_length
nx = int(math.ceil(abs(maxx - minx)/dx))
ny = int(math.ceil(abs(maxy - miny)/dy))

# grid plotting
w = shp.Writer(shp.POLYGON)
w.autoBalance = 1
w.field("ID")
id=0
for i in range(ny):
    for j in range(nx):
        id+=1
        vertices = []
        parts = []
        vertices.append([min(minx+dx*j,maxx),max(maxy-dy*i,miny)])
        vertices.append([min(minx+dx*(j+1),maxx),max(maxy-dy*i,miny)])
        vertices.append([min(minx+dx*(j+1),maxx),max(maxy-dy*(i+1),miny)])
        vertices.append([min(minx+dx*j,maxx),max(maxy-dy*(i+1),miny)])
        parts.append(vertices)
        w.poly(parts)
        w.record(id)
w.save('polygon_grid')

# read data: TBD
grid = gpd.read_file('./polygon_grid.shp')
grid.crs = from_epsg(2263)
uni_bssid = pd.read_pickle("./unique_bssid.p")
uni_bssid = gpd.GeoDataFrame(uni_bssid)
uni_bssid.crs = from_epsg(2263)

# geo points in which cell?
PointInPoly = gpd.sjoin(uni_bssid, grid, how='left', op='intersects')
PointInPoly.dropna(subset=['ID'], inplace=True) # ? why a few points don't intersect with any cell?

# groupby cell.ID to get list of bssid (with duplications) for each cell, then calculate length of unique bssid "uni"
grouped = PointInPoly.groupby('ID').apply(lambda x: reduce(lambda x,y: x+y, x.unique_bssid_list))
bssidInPoly = pd.DataFrame(grouped, columns=['all_bssid_list'])
bssidInPoly['unique_bssid_list'] = map(lambda x: set(x), grouped)
bssidInPoly['cum'] = map(lambda x: len(x), grouped)
bssidInPoly['uni'] = map(lambda x: len(set(x)), grouped)
bssidInPoly['ID'] = bssidInPoly.index
bssidInPoly.reset_index(drop=True, inplace=True)

# merge grid and bssidInPoly
grid_bssid = pd.merge(grid, bssidInPoly, how='left', on='ID')
grid_bssid.to_crs(epsg=2263, inplace=True)
if not delete_empty_cell: 
    grid_bssid.uni.fillna(inplace=True, value=0)
    
# Data for Plot
grid_plot = grid_bssid.loc[:, ['ID', 'uni', 'geometry']]
grid_plot.dropna(subset=['uni'], inplace=True) 
